# Исследование надёжности заёмщиков

# Краткое описание

***Цели проекта:***

Изучить данные по статистике платежеспособности клиента и определить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

***Данные:***

В нашем распоряжении представлен датасет со следующими данными:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

<h1>План проекта<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответы-на-поставленные-вопросы" data-toc-modified-id="Ответы-на-поставленные-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответы на поставленные вопросы</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Выводы 

Подводя общий итог мы получаем картину "добросовестного", "умеренно-добросовестного" и "недобросовестного" заемщика.

Пример "добросовестного" заемщика:
- Нет детей (нет дополнительных доходов)
- Вдовец / в разводе (высокая доля ответственности, умение распоряжаться деньгами и правильно их распределять)
- Высокий уровень дохода
- Кредит на недвижимость

Пример "умеренно-добросовестного" заемщика:
- Есть 1 ребенок (есть затраты, но не такие большие когда детей 2 и больше)
- Женат / гражданский брак
- Низкий уровень дохода
- Кредит на образование или свадьбу

Пример "недобросовестного" заемщика:
- Есть дети
- Не женат 
- Средний уровень дохода
- Кредит на автомобиль

## Изучение данных

In [1]:
import pandas as pd
import numpy as np

from pymystem3 import Mystem
m = Mystem()
from collections import Counter

***Проведем выгрузку датасета с "подстраховкой":***

In [2]:
# подстраховка. Иногда датасеты лежат сразу в корневике, а иногда лежат в папке datasets        
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('data.csv')

display(df.sample(5))
pd.set_option('display.float_format', '{:,.2f}'.format) # избавляемся от лишних запятых
display(df.describe())
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19299,0,-4086.647130,34,среднее,1,женат / замужем,0,M,сотрудник,0,105098.090393,покупка жилья для сдачи
10801,1,-5121.785817,44,среднее,1,гражданский брак,1,F,сотрудник,0,130777.608650,получение дополнительного образования
1853,3,-1848.085867,28,среднее,1,женат / замужем,0,F,сотрудник,0,46039.792927,операции со своей недвижимостью
11855,0,-3714.687881,42,среднее,1,женат / замужем,0,F,компаньон,0,76122.968272,высшее образование
8600,0,-1033.194518,27,среднее,1,гражданский брак,1,F,компаньон,0,229544.185486,на проведение свадьбы


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,"21,525.00","19,351.00","21,525.00","21,525.00","21,525.00","21,525.00","19,351.00"
mean,0.54,"63,046.50",43.29,0.82,0.97,0.08,"167,422.30"
std,1.38,"140,827.31",12.57,0.55,1.42,0.27,"102,971.57"
min,-1.00,"-18,388.95",0.00,0.00,0.00,0.00,"20,667.26"
25%,0.00,"-2,747.42",33.00,1.00,0.00,0.00,"103,053.15"
50%,0.00,"-1,203.37",42.00,1.00,0.00,0.00,"145,017.94"
75%,1.00,-291.10,53.00,1.00,1.00,0.00,"203,435.07"
max,20.00,"401,755.40",75.00,4.00,4.00,1.00,"2,265,604.03"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


***Выводы:***

- Некоторое количество значений в столбцах total_income и days_employed пропущено. Присутствуют отрицательные значения в days_employed, children - это артефакты.

- Можно отметить удивительный показатель максимума в данных о детях. 20 детей это маловероятно, хотя и выполнимо. Необходимо удостовериться у одного-двух заемщиков такое количество детей (что можно принять и расценить как уникальный случай), либо же это ошибка и такие данные есть больше чем у 1-2 заемщиков.

- В трудовом стаже максимальное значение это 401755 дней. При этом создается большой перекос так как даже среднее значение = 63046, а это 172 года! Ошибка, необходимо проверить.

- В столбце о возрасте минимальное значение равно 0, нужно исправить.

Проанализируем столбцы по имеющимся в них уникальным значениям, исправим артефакты и странные значения в данных.

In [3]:
null = df.loc[:, df.columns[df.isna().any()]].isna().sum()
print('Нулевые значения в столбцах: \n{}\n'.format(null.to_string()))

Нулевые значения в столбцах: 
days_employed    2174
total_income     2174



In [4]:
print('Процентное соотношение пропусков относительно общего количества данных:')
100*null/len(df)

Процентное соотношение пропусков относительно общего количества данных:


days_employed   10.10
total_income    10.10
dtype: float64

- Напишем функцию для исследования каждого столбца датафрейма:

In [5]:
for column in df.drop(['days_employed', 'total_income'], axis = 1):
    print('Исследуемый столбец {}'.format(str(column)))
    print('')
    print(df[column].value_counts())
    print('')

Исследуемый столбец children

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Исследуемый столбец dob_years

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Исследуемый столбец education

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ            

***Выводы:***

- В столбце `children` можно заметить артефакт. Скорее всего была допущена ошибка при внесении отрицательного значения (-1) у 47 человек и ошибка у 76 человек при указании количества детей 2, а не 20.
- В столбце `dob_years` можно заметить что у 101 человека возраст равен 0. Несмотря на то что для решения поставленных задач возраст не требуется, сделаем таблицу красивой, заберем 101 у Круэллы. Подставим медианное значение для определенной группы заемщиков в дальнейшем. 
- В столбце `gender` есть неизвестный пол.
- Данные столбца `education, family_status` необходимо привести к нижнему регистру.
- Столбец `debt` содержит логические(булевые) переменные. Его будем использовать при определении зависимостей.
- Столбцы `family_status_id, education_id` являются словарями для столбцов `family_status, education`.
- К столбцу `purpose` необходмо применить лемматизацию и определить основные категории. 

Попробуем найти строку с неизвестными данными по полу и, если есть возможность определить по категории заемщика, а именно по "в декрете" пол. Тогда сможем подправить общую красоту таблицы.

In [6]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,"-2,358.60",24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,"203,905.16",покупка недвижимости


- Пол определить не удалось. Однако поступим как задумывали изначально и определим пол как женский.

- Внесем корректировки в некоторые столбцы:

In [7]:
df['gender'] = df['gender'].replace('XNA', 'F')
df['children'] = df['children'].replace(20, 2).abs()

Высчитаем медиану возраста на каждую группу занятости.

In [8]:
df.groupby('income_type')['dob_years'].median().astype('int')

income_type
безработный        38
в декрете          39
госслужащий        40
компаньон          39
пенсионер          60
предприниматель    42
сотрудник          39
студент            22
Name: dob_years, dtype: int64

Найдем категорию людей у которых встречается в возрасте нулевое значение:

In [9]:
null_age = list(df[df['dob_years'] == 0]['income_type'].unique())
null_age

['пенсионер', 'сотрудник', 'компаньон', 'госслужащий']

Заменим нулевой возраст на медиану в соотвествии с категорией:

In [10]:
for row in null_age:
    median_age = int(df[(df['income_type'] == row) & (df['dob_years'] > 0)]['dob_years'].median())
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == row), 'dob_years'] = median_age

**Вывод**

1. Каждая строка таблицы содержит информацию о клиентах, их платежеспособности, семейном положении, количестве детей и так далее. Что сразу можно заметить, так это отрицательное количество дней трудового стажа и наличие отрицательного количества детей, относится к разряду артефактов. Так как трудовой стаж и наличие детей не могут быть отрицательным, следовательно можно сделать вывод, что при внесении данных произошла техническая ошибка.
Однако, эта ошибка носит человеческий фактор. Дело в том, что несколько лет назад да, могла произойти техническая ошибка при заполнении данных о, например, заемщике. Однако в настоящее время ни одна программа не пропустит такую ошибку, как отрицательные значения.
Значит, делаем вывод, что это была ошибка непосредственно человека, который заполнял эти данные. К примеру, он мог сначала указать дату окончания трудового стажа, а только потом поставить дату начала трудового стажа.

2. Также при применении df.info(), можно заметить пропуски в трудовом стаже и ежемесячном доходе. Помимо этого у них тип float, нужно исправить на тип int.
Столбец о причинах необходимо распределить на категории при помощи лемматизации и привести к нижнему регистру данные из образования.
Исправили данные о детях.
Также была допущена ошибка при заполнении данных о возрасте. Значения не были пропущенны, вместо них стоял 0. Возможно заемщик изначально оставил эту ячейку пустой и до меня кто-то пытался обработать таблицу с пустыми значениями, решив заменить NaN на 0. Поменяли также 0 на медиану в соответсвии с заемщиками.

**Для оценки потенциальных заемщиков необходимо ответить на 4 вопроса:**
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

## Предобработка данных

### Обработка пропусков

In [11]:
df['days_employed'] = df['days_employed'].abs() #выведу модуль, чтобы убрать отрицательные значения

- Согласно с выведенной ранее функцией df.describe(): получается что минимальное значение это 18388 дней, а максимальное 401755. Если мы будем применять среднее значение, то данные окажутся неверны, так как есть большой разброс в числах, да еще при том и со странными значениями трудового стажа в 1100 лет! Поэтому в данном случае мы используем медиану.

In [12]:
df.groupby('income_type').agg({'days_employed' : ['count', 'median']}).reset_index().rename(columns={'count':'all_day', 'median':'median_days'})

income_type days_employed            
                         all_day median_days
0      безработный             2  366,413.65
1        в декрете             1    3,296.76
2      госслужащий          1312    2,689.37
3        компаньон          4577    1,547.38
4        пенсионер          3443  365,213.31
5  предприниматель             1      520.85
6        сотрудник         10014    1,574.20
7          студент             1      578.75

- Очень сильно выделяются медианы по безработынм и пенсионерам. Оценивая данные объективно можно заметить, что, скорее всего, для безработных и пенсионеров указаны данные не в днях, а в часах.

- При помощи индексации выберем строчки с пенсионерами и безработными, и переведем их значения из часов в дни.

In [13]:
df.loc[df['income_type'].isin(['пенсионер', 'безработный']), 'days_employed'] = (df.loc[df['income_type'].isin(['пенсионер', 'безработный']), 'days_employed'])/24

In [14]:
median_employed = df.groupby('income_type').agg({'days_employed' : 'median'})
#сгруппируем данные по столбцу о типах занятости. Применим к столбцу о трудовых днях медиану.

for employed in median_employed.index:
    df.loc[df['income_type'] == employed,'days_employed'] = df.loc[df['income_type'] == employed,'days_employed'].fillna(median_employed.loc[employed, 'days_employed'])

    #df.loc[df['income_type'] == employed,'days_employed'] - это строчки соответствующие типу занятости employed.    
#median_employed.loc позволяет достать числовое значение для конкретного типа, где в .loc первое значение соответствует индексу, а второе столбцу.

In [15]:
df['days_employed'].isna().sum()

0

- Мы заменили все пропущенные значения в трудовом стаже медианами в соответсвии с занятостью и при этом меняем отрицательные значения на положительные. Также меняем данные с безработными и пенсионерами из часов в дни.
Делаем вывод что ни пенсионеры, ни безработные не пытались хитрить. Данные они ввели верно, вот только не в той величине которая была нужна.

- Перейдем к заполнению пропущенных значений о доходах. Предварительно сохраним индексы нужных строк перед заполнением, чтобы в дальнейшем смогли произвести проверку по любому из них.

In [16]:
is_null_rows = df[df.total_income.isnull()].index
is_null_rows

Int64Index([   12,    26,    29,    41,    55,    65,    67,    72,    82,
               83,
            ...
            21415, 21423, 21426, 21432, 21463, 21489, 21495, 21497, 21502,
            21510],
           dtype='int64', length=2174)

In [17]:
df['total_income'] = df['total_income'].abs() #выведу модуль, чтобы убрать отрицательные значения
df.groupby('income_type')['total_income'].median().reset_index() #посчитаю медиану дохода для каждой категории

,income_type,total_income
0,безработный,"131,339.75"
1,в декрете,"53,829.13"
2,госслужащий,"150,447.94"
3,компаньон,"172,357.95"
4,пенсионер,"118,514.49"
5,предприниматель,"499,163.14"
6,сотрудник,"142,594.40"
7,студент,"98,201.63"


In [18]:
median_table = df.groupby('income_type').agg({'total_income' : 'median'})
#сгруппирую данные по столбцу о типах занятости. Применю к столбцу о доходах медиану.

for row in median_table.index:
    df.loc[df['income_type'] == row,'total_income'] = df.loc[df['income_type'] == row,'total_income'].fillna(median_table.loc[row, 'total_income'])

#df.loc[df['income_type'] == inc_type,'total_income'] - это строчки соответствующие типу занятости inc_type.    
#mean_table.loc позволяет достать числовое значение для конкретного типа, где в .loc первое значение соответствует индексу, а второе столбцу.

- Произведем проверку, чтобы убедиться что в пропущенное значение о доходах вставлена соответствующая занятости медиана.

In [19]:
df.loc[12] 

children                           0
days_employed              15,217.22
dob_years                         65
education                    среднее
education_id                       1
family_status       гражданский брак
family_status_id                   1
gender                             M
income_type                пенсионер
debt                               0
total_income              118,514.49
purpose              сыграть свадьбу
Name: 12, dtype: object

- Проверим наличие пропущенных значений.

In [20]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Из проделанной работы можно заметить, что пропущенные значения находились в столбцах о стаже и ежемесячном доходе в количестве 2174 каждый. 

Далее разбор по ежемесячным доходам. Вычислили медиану для каждого типа занятости, затем прописали цикл который автоматически заполняет пропущенные значения в доходах на медианы с соответствующей занятостью. То же самое проделали и с трудовым стажем.

Из медианных значений для трудового стажа видно, что данные значительно завышены для категорий безработные и пенсионеры.

Как и предполагалось в предыдущем выводе, в трудовом стаже скорее всего допущена ошибка человеческого фактора. Поэтому, во-первых, некоторые данные о трудовом стаже записаны с отрицательным значением. Во-вторых, пропуски могут свидетельствовать о том что потенциальный заемщик попросту не пожелал оставлять свои реальные данные о стаже и доходах, возможно он работает неофициально (как известно банки неохотно выдают кредиты самозанятым).

Также, при просмотре всех данных таблицы появляется мысль нужны ли нам столбцы education и education_id / family_status и family_status_id в таком "двойном виде"? Почему не оставить просто education и family_status, избавившись от их айди?

Ответ прост: id - это уникальный идентификатор. Во многих базах данных так формируются таблички: есть где-то отдельный “словарь“, где есть уникальные идентификаторы (education_id) и их расшифровка (education). Чаще всего в таблицах проставляют просто числовой идентификатор, либо для экономии места, либо же, если текст длинный, то чтобы не запутаться и не допустить в ошибку при написании. Если же нужна расшифровка, то смотрят уже в словарик.

### Замена типа данных

- Изучим формат данных:

In [21]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

- Неверный тип у столбцов days_employed, total_income, dob_years.

In [22]:
df['days_employed'] = df['days_employed'].astype('int')

In [23]:
df['total_income'] = df['total_income'].astype('int')

In [24]:
df['dob_years'] = df['dob_years'].astype('int')

### Обработка дубликатов

- Приведем к нижнему регистру данные с образованием и семейным статусом.

In [25]:
df['education'] = df['education'].str.lower()

In [26]:
df['family_status'] = df['family_status'].str.lower()

- Используем метод duplicated()

In [27]:
df[df.duplicated()].sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
18563,0,1574,54,среднее,1,женат / замужем,0,F,сотрудник,0,142594,операции со своей недвижимостью
4851,0,15217,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
9604,0,15217,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
19688,0,15217,61,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с недвижимостью
10697,0,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу


- Уберем дубликаты во всем датафрейме. Однако это не означает что решение правильное, так как вероятность того что эти дубликаты случайны - мала. Следует обратиться к создателям датафрейма для уточнения информации.

In [28]:
df = df.drop_duplicates().reset_index(drop=True)

**Вывод**

В столбце об образовании и семейном статусе можно заметить, что присутствуют данные с заглавными буквами. Чтобы исключить возможные ошибки, приведем все строковые данные таблицы к нижнему регистру.
Также в столбце о детях произвела замену отрицательного значения и 20 детей.
Присутствуют дубликаты во всей таблице, 71 штука - удаляем. Сбрасываем индекс фрейма данных.

### Лемматизация

- В таблице есть столбец `purpose`, в котором описаны цели для получения кредита разными словами.

- Лемматизируем данные. Подсчитаем число слов и их упоминаний в тексте.

In [29]:
df['lemmas'] = df['purpose'].apply(m.lemmatize)
Counter(df['lemmas'].sum()).most_common()

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

- Итог: видно категории образование, автомобиль, жилье(недвижимость) и свадьба. Создадим функцию, которая автоматически будет перебирать все данные в `purpose` и, при помощи лемматизации, переименовывать их для последующей удобной сортировки.

In [30]:
def lemmatization(row):
    lemma = m.lemmatize(row)
    if 'образование' in lemma:
        return 'образование'
    elif 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'свадьба' in lemma:
        return 'свадьба'
    elif ('недвижимость' in lemma) or ('жилье' in lemma):
        return 'недвижимость'
    
df['purpose'] = df['purpose'].apply(lemmatization)

In [31]:
df['purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

**Вывод**

- Изменили данные и распределили их на 4 категории: образование, автомобиль, свадьба и жилье.

### Категоризация данных

- Объединим избранные данные в произвольные группы по заданному критерию. Нужно определить высокий, средний и низкий уровень дохода.
Воспользуемся перцентлями.

In [32]:
# df['total_income'].describe().loc[['25%','50%','75%']]
ranges = np.array(range(25, 100, 25))
df['total_income'].describe(percentiles=ranges/100).loc[[f'{i}%' for i in ranges]]

25%   107,623.00
50%   142,594.00
75%   195,820.25
Name: total_income, dtype: float64

- На основе полученных данных пропишем функцию чтобы распределить `total_income` на низкий (0 - 24%), средний (25 - 74%) и высокий уровень дохода (от 75%).

In [33]:
def income(price):
    if price < 107623:
        return 'низкий уровень дохода'
    elif 107623 <= price < 195820:
        return 'средний уровень дохода'
    else:
        return 'высокий уровень дохода'
    
df['income_group'] = df['total_income'].apply(income)
df['income_group'].value_counts()

средний уровень дохода    10726
высокий уровень дохода     5364
низкий уровень дохода      5364
Name: income_group, dtype: int64

- Также создадим категорию есть дети и нет детей и выведим обновленную талицу.

In [34]:
def children(number):
    if number == 0:
        return 'нет детей'
    else:
        return 'есть дети'
    
df['yes_no_children'] = df['children'].apply(children)
df['yes_no_children'].value_counts()

нет детей    14091
есть дети     7363
Name: yes_no_children, dtype: int64

- Создадим категорию с объединенными данными по женат/замужем и в гражданском браке, а также вдова и разведена, так как у них схожее положение. Выведим обновленную талицу.

Но перед этим выведим словарь 'family_status_id' для 'family_status' (для отработки полученных знаний и чтобы не прописывать каждый семеный статус в условии). Затем удалим дубликаты чтобы новая таблица с данными стала компактной и наглядной.

In [35]:
new_df = df[['family_status', 'family_status_id']]
new_df = new_df.drop_duplicates().reset_index(drop=True)
new_df

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [36]:
def family(status):
    if status == 0 or status == 1:
        return 'женат / замужем / гражданский брак'
    elif status == 2 or status == 3:
        return 'вдовец / вдова / в разводе'
    else:
        return 'не женат / не замужем'
    
df['new_family_status'] = df['family_status_id'].apply(family)
df['new_family_status'].value_counts()

женат / замужем / гражданский брак    16490
не женат / не замужем                  2810
вдовец / вдова / в разводе             2154
Name: new_family_status, dtype: int64

**Вывод**

Выполнили категоризацию по доходу, семейному статусу и наличию детей для последующего ответа на поставленные вопросы.

## Ответы на поставленные вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
correlation_debt_and_children = df.groupby('yes_no_children').agg({'debt' : ['sum', 'count']})
correlation_d_a_c = correlation_debt_and_children['debt']['sum'] / correlation_debt_and_children['debt']['count']
correlation_d_a_c.apply(lambda x: '{:.1%}'.format(x*10)).sort_values(ascending=False)

yes_no_children
есть дети    92.1%
нет детей    75.4%
dtype: object

***Вывод:***

Заемщики у которых есть дети более склонны к задолженностям по возврату кредита. Это может быть связано с дополнительными затратами на детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
correlation_debt_and_family = df.groupby('new_family_status').agg({'debt' : ['sum', 'count']})
correlation_d_a_f = correlation_debt_and_family['debt']['sum'] / correlation_debt_and_family['debt']['count']
correlation_d_a_f.apply(lambda x: '{:.1%}'.format(x*10)).sort_values(ascending=False)

new_family_status
не женат / не замужем                 97.5%
женат / замужем / гражданский брак    80.0%
вдовец / вдова / в разводе            68.7%
dtype: object

***Вывод:***

Согласно полученным данным напрашивается вывод что люди, не состоящие в отношениях и никогда ранее не состоявшие в них, имеют больше задолженностей. Можем предположить что люди, побывавшие или находящиеся на данный момент в браке, знают как правильно распределять деньги и вовремя выделять их на выплаты по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
correlation_debt_and_income = df.groupby('income_group').agg({'debt' : ['sum', 'count']})
correlation_d_a_i = correlation_debt_and_income['debt']['sum'] / correlation_debt_and_income['debt']['count']
correlation_d_a_i.apply(lambda x: '{:.1%}'.format(x*10)).sort_values(ascending=False)

income_group
средний уровень дохода    86.8%
низкий уровень дохода     79.6%
высокий уровень дохода    71.4%
dtype: object

***Вывод:***

На удивление люди с низким уровнем дохода имеют меньший процент задолженностей, нежели чем люди со средним доходом. Это можно объяснить тем что люди с низким доходом привыкли планировать заранее свой бюджет, а потому и процент задолженности у них меньше.

- Как разные цели кредита влияют на его возврат в срок?

In [40]:
correlation_debt_and_purpose = df.groupby('purpose').agg({'debt' : ['sum', 'count']})
correlation_d_a_p = correlation_debt_and_purpose['debt']['sum'] / correlation_debt_and_purpose['debt']['count']
correlation_d_a_p.apply(lambda x: '{:.1%}'.format(x*10)).sort_values(ascending=False)

purpose
автомобиль      93.6%
образование     92.2%
свадьба         80.0%
недвижимость    72.3%
dtype: object

***Вывод:***

Больше всего задолженностей у людей с целями кредита по автомобилям и образованию. Самый низкий процент задолженностей у людей с целью купить недвижимость. Возможно что люди твердо оценивают свои силы и потому на покупкку недвижимости заранее выделяют нужные деньги. В то время как покупка автомобиля зачастую является относительно спонтанным решением. Кредит на образование тоже можно оценивать как спонтанный, так как баллы по итоговым школьным экзаменам могут не оправдать ожидание и по итогу вместо бюджета придется оплачивать платное образование. К такой ситуации зачастую родители оказываются не готовы, а потому заранее не планируют как будут отдавать деньги и соглашаются на любые условия по кредиту.

## Выводы 

Подводя общий итог мы получаем картину "добросовестного", "умеренно-добросовестного" и "недобросовестного" заемщика.

Пример "добросовестного" заемщика:
- Нет детей (нет дополнительных доходов)
- Вдовец / в разводе (высокая доля ответственности, умение распоряжаться деньгами и правильно их распределять)
- Высокий уровень дохода
- Кредит на недвижимость

Пример "умеренно-добросовестного" заемщика:
- Есть 1 ребенок (есть затраты, но не такие большие когда детей 2 и больше)
- Женат / гражданский брак
- Низкий уровень дохода
- Кредит на образование или свадьбу

Пример "недобросовестного" заемщика:
- Есть дети
- Не женат 
- Средний уровень дохода
- Кредит на автомобиль